In [1]:
!pip install albumentations
!pip install torch

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
ROOT = '/content/drive/My Drive/Western_blot'
SEC = '/content/drive/My Drive/Augmented_Western_blot'

In [19]:
import os
import numpy as np
from PIL import Image
def load_images(root_folder):
    # Array to store loaded images
    images = []
    for filename in os.listdir(root_folder):
        filepath = os.path.join(root_folder, filename)
        if os.path.isfile(filepath):
            # Open the image file using PIL and convert it to a numpy array
            image = np.array(Image.open(filepath))
            # Append the numpy array representing the image to the list of images
            images.append(image)
    return images
    
    # images = []
    #     filepath = os.path.join(root_folder, filename)
    #     image = np.array(Image.open(filepath))
    #     images.append(np.array(image))
    # return images

In [22]:
images = load_images(ROOT)

In [29]:
import torch
import torchvision.transforms as transforms
from torch.utils.data import Dataset
from PIL import Image

class CustomImageDataset(Dataset):
    def __init__(self, image_list, labels, transform=None):
        self.image_list = image_list
        self.transform = transform
        self.labels = labels

    def __len__(self):
        return len(self.image_list)

    def __getitem__(self, idx, label):
        print(idx)
        # Path of image at index
        image_path = self.image_list[idx]
        # Open the image and convert it to RGB mode
        image = Image.open(image_path).convert('RGB')
        if self.transform:
            image = self.transform(image)
        # Get the label corresponding to the image
        label = self.labels[idx]
        return image, label

transform = transforms.Compose([
    # Resizi image
    transforms.Resize((224, 224)),
    # Convert image to PyTorch tensor
    transforms.ToTensor(),
    # Normalize the image
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])
# Create instance of CustomImageDataset class with the transformation
dataset = CustomImageDataset(images, transform=transform)



TypeError: CustomImageDataset.__init__() missing 1 required positional argument: 'labels'

In [28]:
import torch
import torchvision
import torchvision.transforms as transforms
from torchvision import models

dataloader = torch.utils.data.DataLoader(dataset, batch_size=32, shuffle=True)

resnet = models.resnet50(pretrained=True)

for param in resnet.parameters():
    param.requires_grad = False

num_classes = 5

class CustomClassifier(torch.nn.Module):
    def __init__(self, in_features, num_classes):
        super(CustomClassifier, self).__init__()
        self.fc1 = torch.nn.Linear(in_features, 512)
        self.fc2 = torch.nn.Linear(512, num_classes)
        self.relu = torch.nn.ReLU()
        self.dropout = torch.nn.Dropout(0.5)

    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)
        return x

resnet.fc = CustomClassifier(resnet.fc.in_features, num_classes)

for param in resnet.fc.parameters():
    param.requires_grad = True

criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(resnet.parameters(), lr=0.001)

num_epochs = 10
for epoch in range(num_epochs):
    for images, labels in dataloader:
        optimizer.zero_grad()
        outputs = resnet(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

torch.save(resnet.state_dict(), 'resnet_model.pth')


/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


AttributeError: 'numpy.ndarray' object has no attribute 'read'